Binary Bank Competition - Kaggle


### Jupyter Notebook in VS Code

From directory with virtual environment in it, use this terminal command to use Jupyter Notebook in VS Code:

jupyter notebook --NotebookApp.allow_origin='*' --NotebookApp.ip='0.0.0.0'

### Select Python Interpreter

Open Command Palette as follows: Ctrl+Shift+P

Then select from VS Code Command Palette: "Python: Select Interpreter" and choose the same Python interpreter that you used to install Jupyter.

If server not shown, enter server manually as follows: 
http://localhost:8888

Reminder:  pw: ""

In [21]:
from datetime import datetime

import numpy as np
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter

from sklearn.datasets import make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from stepbystep_v0 import StepByStep

# Binary Classification Problem based on Bank Customer Marketing Call Data

# Model Training

### Load Training Data

In [22]:
# Import data from .csv files
raw_train_data = pd.read_csv('binary_bank_comp/dataset/train.csv')

# Display the first few rows of the dataset
print(raw_train_data[:5])

   id  age          job  marital  education default  balance housing loan  \
0   0   42   technician  married  secondary      no        7      no   no   
1   1   38  blue-collar  married  secondary      no      514      no   no   
2   2   36  blue-collar  married  secondary      no      602     yes   no   
3   3   27      student   single  secondary      no       34     yes   no   
4   4   26   technician  married  secondary      no      889     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0  cellular   25   aug       117         3     -1         0  unknown  0  
1   unknown   18   jun       185         1     -1         0  unknown  0  
2   unknown   14   may       111         2     -1         0  unknown  0  
3   unknown   28   may        10         2     -1         0  unknown  0  
4  cellular    3   feb       902         1     -1         0  unknown  1  


### Split the train dataset

In [23]:
# Remove id column
raw_train_data.drop(columns=['id'], inplace=True)

# Split the dataset into features and target variable
X = raw_train_data.drop(columns=['y'])
y = raw_train_data['y']

# Split the dataset into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Print the shapes of the resulting datasets
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)

# Print the first few rows of the training set
print("Training set preview:")
print(X_train.head())
print(y_train.head())

Training set shape: (675000, 16) (675000,)
Validation set shape: (75000, 16) (75000,)
Training set preview:
        age          job   marital  education default  balance housing loan  \
114614   34       admin.  divorced  secondary      no        0     yes   no   
581446   35  blue-collar   married  secondary      no      622     yes   no   
144124   37   management   married   tertiary      no       16      no   no   
715271   45   management   married   tertiary      no     8897      no   no   
226111   41   management    single   tertiary      no      491      no   no   

         contact  day month  duration  campaign  pdays  previous poutcome  
114614  cellular   13   may       574         3    176         2    other  
581446  cellular   17   apr        91         1     -1         0  unknown  
144124  cellular    9   feb       126         4     -1         0  unknown  
715271  cellular   10   sep       215         1    185         7  failure  
226111  cellular   13   aug       119

### Data Cleaning

In [24]:
# Clean the dataset

# Identify numerical columns (excluding the target and any non-feature columns)
X_train_numerical_cols = X_train.select_dtypes(include=['int64', 'float64'])
X_val_numerical_cols = X_val.select_dtypes(include=['int64', 'float64'])

# Identify categorical columns
X_train_categorical_cols = X_train.select_dtypes(include=['object', 'category'])
X_val_categorical_cols = X_val.select_dtypes(include=['object', 'category'])

# Convert categorical features to numerical using one-hot encoding using 1s and 0s
X_train_categorical_encoded = pd.get_dummies(X_train_categorical_cols, drop_first=False).astype(int)
X_val_categorical_encoded = pd.get_dummies(X_val_categorical_cols, drop_first=False).astype(int)

# Combine the numerical and categorical features
X_train_combined = pd.concat([X_train_numerical_cols, X_train_categorical_encoded], axis=1)
X_val_combined = pd.concat([X_val_numerical_cols, X_val_categorical_encoded], axis=1)

# Numeric Feature Standardization
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train_combined)
X_val_scaled = sc.transform(X_val_combined)

# Convert the scaled arrays back to DataFrames for easier handling
X_train = pd.DataFrame(X_train_scaled, columns=X_train_combined.columns)
X_val = pd.DataFrame(X_val_scaled, columns=X_val_combined.columns)

# Show the cleaned dataset
print(X_train)
print(X_val)

             age   balance       day  duration  campaign     pdays  previous  \
0      -0.684975 -0.424858 -0.377990  1.166370  0.156123  1.985740  1.271530   
1      -0.585930 -0.205250  0.106778 -0.606237 -0.579892 -0.302930 -0.222930   
2      -0.387839 -0.419209 -0.862757 -0.477787  0.524131 -0.302930 -0.222930   
3       0.404524  2.716381 -0.741565 -0.151158 -0.579892  2.102114  5.007681   
4       0.008342 -0.251502 -0.377990 -0.503477  0.524131 -0.302930 -0.222930   
...          ...       ...       ...       ...       ...       ...       ...   
674995 -0.982112 -0.221138  1.439888 -0.492467  0.524131  2.050392  1.271530   
674996 -0.982112 -0.424858  1.439888 -0.617247  0.156123 -0.302930 -0.222930   
674997 -1.081157  0.005178  0.955121 -0.764047 -0.579892 -0.302930 -0.222930   
674998  0.305478 -0.257857  1.803464 -0.657617 -0.211885 -0.302930 -0.222930   
674999  1.394978 -0.424858 -1.347525 -0.598897 -0.579892 -0.302930 -0.222930   

        job_admin.  job_blue-collar  jo

### Data Preparation

In [25]:
torch.manual_seed(13)

# Convert pandas DataFrames to Numpy arrays
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()

# Builds tensors from numpy arrays
x_train_tensor = torch.as_tensor(X_train).float()
x_val_tensor = torch.as_tensor(X_val).float()
# Reshapes y_train and y_val to be column vectors
y_train_tensor = torch.as_tensor(y_train.reshape(-1, 1)).float()
y_val_tensor = torch.as_tensor(y_val.reshape(-1, 1)).float()

# Builds dataset containing ALL data points
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

# Builds a loader of each set
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=16)


## Model

$$
\Large
y = b + w_1x_1 + w_2x_2 + ... + w_nx_n + \epsilon
$$

$$
\Large 
y =
\begin{cases}
1,\ \text{if }b + w_1x_1 + w_2x_2 + ... + w_nx_n \ge 0
\\
0,\ \text{if }b + w_1x_1 + w_2x_2 + ... + w_nx_n < 0
\end{cases}
$$

### Logits

$$
\Large
z = b + w_1x_1 + w_2x_2 + ... + w_nx_n
$$

### Probabilities

$$
\Large
\begin{aligned}
& \text{P}(y=1) \approx 1.0, & \text{if } &z \gg 0
\\
& \text{P}(y=1) = 0.5, & \text{if } &z = 0
\\
& \text{P}(y=1) \approx 0.0, & \text{if } &z \ll 0
\end{aligned}
$$

### From Logits to Probabilities

$$
\Large
\begin{aligned}
b + w_1x_1 + w_2x_2 + ... + w_nx_n = &\ z = \text{log}\left(\frac{p}{1-p}\right) \nonumber
\\
e^{b + w_1x_1 + w_2x_2 + ... + w_nx_n} = &\ e^z = \frac{p}{1-p} \nonumber
\end{aligned}
$$

$$
\Large
\begin{aligned}
\frac{1}{e^z}& = \frac{1-p}{p}
\\
e^{-z}& = \frac{1}{p} - 1
\\
1 + e^{-z}& = \frac{1}{p}&
\\
p& = \frac{1}{1 + e^{-z}}
\end{aligned}
$$

$$
\Large
p = \sigma(z) = \frac{1}{1+e^{-z}}
$$

### Logistic Regression

$$
\Large
\text{P}(y=1) = \sigma(z) = \sigma(b+w_1x_1+w_2x_2 + ... + w_nx_n) = \frac{1}{1+e^{-(b+w_1x_1+w_2x_2 + ... + w_nx_n)}}      
$$

### A Note on Notation (for reference)

$$
\Large W =
\underset{(3 \times 1)}{
\begin{bmatrix}
b \\
w_1 \\
w_2
\end{bmatrix}};
X = 
\underset{(3 \times 1)}{
\begin{bmatrix}
1 \\
x_1 \\
x_2
\end{bmatrix}}
$$

$$
\Large 
\begin{aligned}
z
& = W^T X
=
\underset{(1 \times 3)}{
\begin{bmatrix}
- & w^{T} & -\\
\end{bmatrix}}
\underset{(3 \times 1)}{
\begin{bmatrix}
1 \\
x_1 \\
x_2
\end{bmatrix}}
= \underset{(1 \times 3)}{
\begin{bmatrix}
b & w_1 & w_2
\end{bmatrix}}
\underset{(3 \times 1)}{
\begin{bmatrix}
1 \\
x_1 \\
x_2
\end{bmatrix}}\\
& = b + w_1x_1 + w_2x_2
\end{aligned}
$$

## Loss

$$
\Large y_i = 1 \Rightarrow \text{error}_i=\text{log}(\text{P}(y_i=1))
$$

$$
\Large \text{P}(y_i=0)=1-\text{P}(y_i=1)
$$

$$
\Large y_i = 0 \Rightarrow \text{error}_i=\text{log}(1-\text{P}(y_i=1))
$$

$$
\Large
\text{BCE}(y)={-\frac{1}{(N_{\text{pos}}+N_{\text{neg}})}\Bigg[{\sum_{i=1}^{N_{\text{pos}}}{\text{log}(\text{P}(y_i=1))} + \sum_{i=1}^{N_{\text{neg}}}{\text{log}(1 - \text{P}(y_i=1))}}\Bigg]}
$$

$$
\Large
\text{BCE}(y)={-\frac{1}{N}\sum_{i=1}^{N}{\left[y_i \text{log}(\text{P}(y_i=1)) + (1-y_i) \text{log}(1-\text{P}(y_i=1))\right]}}
$$

### Model Configuration

In [26]:
# Sets learning rate - "eta" ~ the "n" like Greek letter
lr = 0.001

torch.manual_seed(42)
feature_cnt = X_train.shape[1]
print("Number of features:", feature_cnt)
model = nn.Sequential()
model.add_module('linear0', nn.Linear(feature_cnt, 10))
model.add_module('activation0', nn.ReLU())
model.add_module('linear2', nn.Linear(10, 3))
model.add_module('activation2', nn.ReLU())
model.add_module('output', nn.Linear(3, 1))

# Defines a SGD optimizer to update the parameters
optimizer = optim.SGD(model.parameters(), lr=lr)

# Defines an Adam optimizer to update the parameters
# optimizer = optim.Adam(model.parameters(), lr=lr)

# Defines a BCE loss function
loss_fn = nn.BCEWithLogitsLoss()

Number of features: 51


### Model Definition

In [27]:
sbs = StepByStep(model=model, loss_fn=loss_fn, optimizer=optimizer)

### Model Training

In [ ]:
n_epochs = 250

sbs.set_loaders(train_loader, val_loader)
sbs.set_tensorboard('run2', folder='binary_bank_comp/runs')
sbs.train(n_epochs)

In [ ]:
fig = sbs.plot_losses()

In [ ]:
print(model.state_dict())

In [36]:
# Save model
filename='binary_bank_comp/checkpoint_sgd_3layer_lr005_250.pth'
sbs.save_checkpoint(filename=filename)

## Inference

### Load Test Data

In [28]:
# Import data from .csv file
raw_test_data = pd.read_csv('binary_bank_comp/dataset/test.csv')

# Display the first few rows of the dataset
print(raw_test_data[:5])

       id  age            job  marital  education default  balance housing  \
0  750000   32    blue-collar  married  secondary      no     1397     yes   
1  750001   44     management  married   tertiary      no       23     yes   
2  750002   36  self-employed  married    primary      no       46     yes   
3  750003   58    blue-collar  married  secondary      no    -1380     yes   
4  750004   28     technician   single  secondary      no     1950     yes   

  loan   contact  day month  duration  campaign  pdays  previous poutcome  
0   no   unknown   21   may       224         1     -1         0  unknown  
1   no  cellular    3   apr       586         2     -1         0  unknown  
2  yes  cellular   13   may       111         2     -1         0  unknown  
3  yes   unknown   29   may       125         1     -1         0  unknown  
4   no  cellular   22   jul       181         1     -1         0  unknown  


### Remove and save id column

In [29]:
# Remove and save id column
id_column = raw_test_data['id']
raw_test_data.drop(columns=['id'], inplace=True)
id_column

0         750000
1         750001
2         750002
3         750003
4         750004
           ...  
249995    999995
249996    999996
249997    999997
249998    999998
249999    999999
Name: id, Length: 250000, dtype: int64

### Clean Test Data

In [30]:
# Clean the test dataset

# Identify numerical columns (excluding the target and any non-feature columns)
X_test_numerical_cols = raw_test_data.select_dtypes(include=['int64', 'float64'])

# Identify categorical columns
X_test_categorical_cols = raw_test_data.select_dtypes(include=['object', 'category'])

# Convert categorical features to numerical using one-hot encoding using 1s and 0s
X_test_categorical_encoded = pd.get_dummies(X_test_categorical_cols, drop_first=False).astype(int)

# Combine the numerical and categorical features
X_test_combined = pd.concat([X_test_numerical_cols, X_test_categorical_encoded], axis=1)

# Numeric Feature Standardization
# Note: sc = StandardScaler() from training data preprocessing fit on X_train_combined
X_test_scaled = sc.transform(X_test_combined)

# Convert the scaled arrays back to DataFrames for easier handling
X_test = pd.DataFrame(X_test_scaled, columns=X_test_combined.columns)

# Show the cleaned dataset
print(X_test)

             age   balance       day  duration  campaign     pdays  previous  \
0      -0.883066  0.068377  0.591545 -0.118128 -0.579892 -0.302930 -0.222930   
1       0.305478 -0.416737 -1.589908  1.210410 -0.211885 -0.302930 -0.222930   
2      -0.486885 -0.408617 -0.377990 -0.532837 -0.211885 -0.302930 -0.222930   
3       1.692114 -0.912090  1.561080 -0.481457 -0.579892 -0.302930 -0.222930   
4      -1.279248  0.263623  0.712737 -0.275938 -0.579892 -0.302930 -0.222930   
...          ...       ...       ...       ...       ...       ...       ...   
249995  0.206433 -0.424858  0.227969 -0.701657 -0.211885 -0.302930 -0.222930   
249996 -0.090703 -0.240557  0.349161  1.008560 -0.579892  2.153835  0.524300   
249997  2.187341 -0.413207 -1.589908 -0.286948 -0.579892  0.899592  5.754912   
249998  0.899751  0.503356  1.682272 -0.341998 -0.211885 -0.302930 -0.222930   
249999 -1.180202 -0.169943 -1.226333 -0.503477 -0.579892 -0.302930 -0.222930   

        job_admin.  job_blue-collar  jo

### Data Preparation

In [31]:
torch.manual_seed(13) # Used for reproducibility of shuffle in DataLoader

# Convert pandas DataFrames to Numpy arrays
X_test = X_test.to_numpy()

# Builds tensors from numpy arrays
x_test_tensor = torch.as_tensor(X_test).float()

# Builds dataset containing ALL data points
test_dataset = TensorDataset(x_test_tensor)

# Builds a loader of each set
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=True)

### Model Configuration - defined in training section

### Model Definition - defined in training section

### Load Trained Model

In [ ]:
# Load trained model using filename defined in training section
# filename='binary_bank_comp/checkpoint_sgd_3layer_lr005_250.pth'
sbs.load_checkpoint(filename=filename)

### Making Predictions (Logits)

In [38]:
predictions = sbs.predict(x_test_tensor)
predictions

array([[-6.8138494],
       [-1.3085092],
       [-7.91028  ],
       ...,
       [ 1.1134808],
       [-6.773426 ],
       [-1.4504104]], shape=(250000, 1), dtype=float32)

#### Making Predictions (Probabilities)

In [39]:
# Convert predictions to probabilities using the sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

probabilities = sigmoid(predictions)  # Use torch's built-in sigmoid for numerical stabilit
probabilities

array([[1.0972511e-03],
       [2.1273640e-01],
       [3.6681708e-04],
       ...,
       [7.5277746e-01],
       [1.1424626e-03],
       [1.8993843e-01]], shape=(250000, 1), dtype=float32)

#### Making Predictions (Classes)

$$
\Large y =
\begin{cases}
1,\ \text{if P}(y=1) \ge 0.5
\\
0,\ \text{if P}(y=1) < 0.5
\end{cases}
$$

$$
\Large y =
\begin{cases}
1,\ \text{if } \sigma(z) \ge 0.5
\\
0,\ \text{if } \sigma(z) < 0.5
\end{cases}
$$

$$
\Large y =
\begin{cases}
1,\ \text{if } z \ge 0
\\
0,\ \text{if } z < 0
\end{cases}
$$

In [40]:
classes = (predictions >= 0).astype(int)

# Count positive predictions
positive_count = (classes == 1).sum()

output_classes = classes.astype(int).flatten()
output_probabilities = probabilities.astype(float).flatten()

# Get current date and time as a string, e.g., '2025-08-13_15-30-45'
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# Output index ids and classes to .csv file
classes_df = pd.DataFrame({'id': id_column, 'class': output_classes})
classes_filename = f'binary_bank_comp/results/predicted_classes_{timestamp}.csv'
classes_df.to_csv(classes_filename, index=False)

# Output index ids and probabilities to .csv file
probabilities_df = pd.DataFrame({'id': id_column, 'y': output_probabilities})
probabilities_filename = f'binary_bank_comp/results/predicted_probabilities_{timestamp}.csv'
probabilities_df.to_csv(probabilities_filename, index=False)

print(id_column)
print(output_classes)
print(output_probabilities)
print()
print(positive_count)

0         750000
1         750001
2         750002
3         750003
4         750004
           ...  
249995    999995
249996    999996
249997    999997
249998    999998
249999    999999
Name: id, Length: 250000, dtype: int64
[0 0 0 ... 1 0 0]
[1.09725108e-03 2.12736398e-01 3.66817083e-04 ... 7.52777457e-01
 1.14246260e-03 1.89938426e-01]

26559


# Model Evaluation - WIP

## Determine confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = (classes > 0).float()
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
logits_val = sbs.predict(X_val)
probabilities_val = sigmoid(logits_val).squeeze()
threshold = 0.5

In [ ]:
def probability_line(ax, y, probs, threshold, shift=0.0, annot=False, colors=None):
    if colors is None:
        colors = ['r', 'b']
    ax.grid(False)
    ax.set_ylim([-.1, .1])
    ax.axes.get_yaxis().set_visible(False)
    ax.plot([0, 1], [0, 0], linewidth=2, c='k', zorder=1)
    ax.plot([0, 0], [-.1, .1], c='k', zorder=1)
    ax.plot([1, 1], [-.1, .1], c='k', zorder=1)

    tn = (y == 0) & (probs < threshold)
    fn = (y == 0) & (probs >= threshold)
    tp = (y == 1) & (probs >= threshold)
    fp = (y == 1) & (probs < threshold)

    ax.plot([threshold, threshold], [-.1, .1], c='k', zorder=1, linestyle='--')
    ax.scatter(probs[tn], np.zeros(tn.sum()) + shift, c=colors[0], s=150, zorder=2, edgecolor=colors[0], linewidth=3)
    ax.scatter(probs[fn], np.zeros(fn.sum()) + shift, c=colors[0], s=150, zorder=2, edgecolor=colors[1], linewidth=3)

    ax.scatter(probs[tp], np.zeros(tp.sum()) - shift, c=colors[1], s=150, zorder=2, edgecolor=colors[1], linewidth=3)
    ax.scatter(probs[fp], np.zeros(fp.sum()) - shift, c=colors[1], s=150, zorder=2, edgecolor=colors[0], linewidth=3)

    ax.set_xlabel(r'$\sigma(z) = P(y=1)$')
    ax.set_title('Threshold = {}'.format(threshold))

    if annot:
        ax.annotate('TN', xy=(.20, .03), c='k', weight='bold', fontsize=20)
        ax.annotate('FN', xy=(.20, -.08), c='k', weight='bold', fontsize=20)
        ax.annotate('FP', xy=(.70, .03), c='k', weight='bold', fontsize=20)
        ax.annotate('TP', xy=(.70, -.08), c='k', weight='bold', fontsize=20)
    return ax

def probability_contour(ax, model, device, X, y, threshold, cm=None, cm_bright=None):
    if cm is None:
        cm = plt.cm.RdBu
    if cm_bright is None:
        cm_bright = ListedColormap(['#FF0000', '#0000FF'])

    h = .02  # step size in the mesh

    x_min, x_max = -2.25, 2.25
    y_min, y_max = -2.25, 2.25

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    logits = model(torch.as_tensor(np.c_[xx.ravel(), yy.ravel()]).float().to(device))
    logits = logits.detach().cpu().numpy().reshape(xx.shape)

    yhat = sigmoid(logits)

    ax.contour(xx, yy, yhat, levels=[threshold], cmap="Greys", vmin=0, vmax=1)
    contour = ax.contourf(xx, yy, yhat, 25, cmap=cm, alpha=.8, vmin=0, vmax=1)
    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright, edgecolors='k')
    # Plot the testing points
    #ax.scatter(X_val[:, 0], X_val[:, 1], c=y_val, cmap=cm_bright, edgecolors='k', alpha=0.6)

    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel(r'$X_1$')
    ax.set_ylabel(r'$X_2$')
    ax.set_title(r'$\sigma(z) = P(y=1)$')
    ax.grid(False)

    ax_c = plt.colorbar(contour)
    ax_c.set_ticks([0, .25, .5, .75, 1])
    return ax

def figure9(x, y, model, device, probabilities, threshold, shift=0.0, annot=False, cm=None, cm_bright=None):
    fig = plt.figure(figsize=(15, 5))
    gs = fig.add_gridspec(3, 3)

    ax = fig.add_subplot(gs[:, 0])
    probability_contour(ax, model, device, x, y, threshold, cm, cm_bright)
    
    if cm_bright is None:
        colors = ['r', 'b']
    else:
        colors = cm_bright.colors

    ax = fig.add_subplot(gs[1, 1:])
    probability_line(ax, y, probabilities, threshold, shift, annot, colors)

    fig.tight_layout()
    return fig

fig = figure9(X_val, y_val, sbs.model, sbs.device, probabilities_val, threshold)

In [ ]:
fig = figure10(y_val, probabilities_val, threshold, 0.04, False)

### Confusion Matrix

In [ ]:
fig = figure10(y_val, probabilities_val, threshold, 0.04, True)

In [ ]:
cm_thresh50 = confusion_matrix(y_val, (probabilities_val >= .5))
cm_thresh50

#### True and False Positives and Negatives

In [ ]:
def split_cm(cm):
    # Actual negatives go in the top row, 
    # above the probability line
    actual_negative = cm[0]
    # Predicted negatives go in the first column
    tn = actual_negative[0]
    # Predicted positives go in the second column
    fp = actual_negative[1]

    # Actual positives go in the bottow row, 
    # below the probability line
    actual_positive = cm[1]
    # Predicted negatives go in the first column
    fn = actual_positive[0]
    # Predicted positives go in the second column
    tp = actual_positive[1]
    
    return tn, fp, fn, tp

### Metrics

### True and False Positive Rates

$$
\Large \text{TPR} = \frac{\text{TP}}{\text{TP + FN}} \ \ \  \text{FPR} = \frac{\text{FP}}{\text{FP + TN}}
$$

In [ ]:
def tpr_fpr(cm):
    tn, fp, fn, tp = split_cm(cm)
    
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    
    return tpr, fpr

In [ ]:
tpr_fpr(cm_thresh50)

### Precision and Recall

$$
\Large \text{Recall} = \frac{\text{TP}}{\text{TP + FN}} \ \ \  \text{Precision} = \frac{\text{TP}}{\text{TP + FP}}
$$

In [ ]:
def precision_recall(cm):
    tn, fp, fn, tp = split_cm(cm)
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    
    return precision, recall

In [ ]:
precision_recall(cm_thresh50)

### Accuracy

$$
\Large \text{Accuracy} = \frac{\text{TP+TN}}{\text{TP+TN+FP+FN}}
$$

## Trade-offs and Curves

In [ ]:
fig = eval_curves_from_probs(y_val, probabilities_val, [.5], annot=True)

### Low Threshold

In [ ]:
fig = figure9(X_val, y_val, sbs.model, sbs.device, probabilities_val, threshold=0.3, shift=0.04, annot=True)

In [ ]:
confusion_matrix(y_val, (probabilities_val >= 0.3))

In [ ]:
fig = eval_curves_from_probs(y_val, probabilities_val, [.3, .5], annot=True)

### High Threshold

In [ ]:
fig = figure9(X_val, y_val, sbs.model, sbs.device, probabilities_val, threshold=0.7, shift=0.04, annot=True)

In [ ]:
confusion_matrix(y_val, (probabilities_val >= 0.7))

In [ ]:
fig = eval_curves_from_probs(y_val, probabilities_val, [.3, .5, .7], annot=True)

### ROC and PR Curves

In [ ]:
threshs = np.linspace(0.,1,11)

In [ ]:
fig = figure17(y_val, probabilities_val, threshs)

In [ ]:
fpr, tpr, thresholds1 = roc_curve(y_val, probabilities_val)
prec, rec, thresholds2 = precision_recall_curve(y_val, probabilities_val)

In [ ]:
fig = eval_curves(fpr, tpr, rec, prec, thresholds1, thresholds2, line=True)

### The Precision Quirk

In [ ]:
fig = figure19(y_val, probabilities_val)

$$
\Large \text{Precision}(\text{thresh}=0.40)=\frac{13}{13+2}=0.8666
$$

$$
\Large \text{Precision}(\text{thresh}=0.50)=\frac{(13-1)}{(13-1)+2}=\frac{12}{12+2}=0.8571
$$

$$
\Large \text{Precision}(\text{thresh}=0.57)=\frac{12}{12+(2-1)}=\frac{12}{12+1}=0.9230
$$

### Best and Worst Curves

In [ ]:
# Best
fig = figure20(y_val)

In [ ]:
np.random.seed(39)
random_probs = np.random.uniform(size=y_val.shape)

In [ ]:
fpr_random, tpr_random, thresholds1_random = roc_curve(y_val, random_probs)
prec_random, rec_random, thresholds2_random = precision_recall_curve(y_val, random_probs)

In [ ]:
# Worst
fig = figure21(y_val, random_probs)

### Comparing Models

In [ ]:
# Area under the curves of our model
auroc = auc(fpr, tpr)
aupr = auc(rec, prec)
print(auroc, aupr)

In [ ]:
# Area under the curves of the random model
auroc_random = auc(fpr_random, tpr_random)
aupr_random = auc(rec_random, prec_random)
print(auroc_random, aupr_random)

## Putting It All Together

In [ ]:
torch.manual_seed(13)

# Builds tensors from numpy arrays
x_train_tensor = torch.as_tensor(X_train).float()
y_train_tensor = torch.as_tensor(y_train.reshape(-1, 1)).float()

x_val_tensor = torch.as_tensor(X_val).float()
y_val_tensor = torch.as_tensor(y_val.reshape(-1, 1)).float()

# Builds dataset containing ALL data points
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

# Builds a loader of each set
train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=16, 
    shuffle=True
)
val_loader = DataLoader(dataset=val_dataset, batch_size=16)

In [ ]:
# Sets learning rate - this is "eta" ~ the "n" like Greek letter
lr = 0.1

torch.manual_seed(42)
model = nn.Sequential()
model.add_module('linear', nn.Linear(2, 1))

# Defines a SGD optimizer to update the parameters
optimizer = optim.SGD(model.parameters(), lr=lr)

# Defines a BCE loss function
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
n_epochs = 100

sbs = StepByStep(model, loss_fn, optimizer)
sbs.set_loaders(train_loader, val_loader)
sbs.train(n_epochs)

In [ ]:
print(model.state_dict())

In [ ]:
logits_val = sbs.predict(X_val)
probabilities_val = sigmoid(logits_val).squeeze()
cm_thresh50 = confusion_matrix(y_val, (probabilities_val >= 0.5))
cm_thresh50